In [ ]:
# Load Apple Health Data in to CAS for use with SAS Visusal Statistics

In [517]:
import xml.etree.ElementTree as et
import pandas as pd
import numpy as np
from datetime import *
import matplotlib.pyplot as plt
import re 
%matplotlib inline
plt.rcParams['figure.figsize'] = 16, 8

# Load the tables into CAS using SAS SWAT 
You will need access to a SAS Viya cluster for this part of the program to work. The next cell establishes a connection to the cluster, and the subsequent cells are a combination of locally and remotely executed code. 

In [ ]:
from swat import *
cashost='localhost'
casport=5570
casauth='~/.authinfo'
sess = CAS(cashost, casport, authinfo=casauth, caslib="casuser")

In [35]:
# list available caslibs
sess.caslibinfo()

,Name,Type,Description,Path,Definition,Subdirs,Local,Active,Personal,Hidden
0,CASUSER(sasdemo),PATH,Personal File System Caslib,/home/sasdemo/casuser/,,1.0,0.0,1.0,1.0,0.0
1,DemoData,PATH,"DemoCenter Shared and writeable caslib, access...",/opt/sasinside/DemoData/,,0.0,0.0,0.0,0.0,0.0
2,Formats,PATH,Stores user defined formats.,/opt/sas/viya/config/data/cas/default/formats/,,1.0,0.0,0.0,0.0,0.0
3,Models,PATH,Stores models created by Visual Analytics for ...,/opt/sas/viya/config/data/cas/default/models/,,0.0,0.0,0.0,0.0,0.0
4,Public,PATH,"Shared and writeable caslib, accessible to all...",/opt/sas/viya/config/data/cas/default/public/,,1.0,0.0,0.0,0.0,0.0
5,SystemData,PATH,"Stores application generated data, used for ge...",/opt/sas/viya/config/data/cas/default/sysData/,,0.0,0.0,0.0,0.0,0.0


In [36]:
# Load necessary actionsets
sess.loadactionset(actionset="table")

NOTE: Added action set 'table'.


[actionset]

 'table'

+ Elapsed: 0.00118s, user: 0.001s, mem: 0.0595mb

In [46]:
HR_df = xmltodf("HKQuantityTypeIdentifierHeartRate","HeartRate")
SC_df = xmltodf("HKQuantityTypeIdentifierStepCount","StepCount")

,Create,StepCount,Month,Day,Hour
0,2016-06-30 20:38:47-04:00,13,2016-06,30,20
1,2016-06-30 22:21:39-04:00,59,2016-06,30,22
2,2016-06-30 23:11:11-04:00,146,2016-06,30,23
3,2016-07-01 07:23:59-04:00,35,2016-07,01,07
4,2016-07-01 11:31:15-04:00,22,2016-07,01,11


In [50]:
def tbltoCAS(tbl_ld,casName):
    if not sess.table.tableExists(table=casName).exists:
        upRes = sess.upload(data=tbl_ld,casout=casName)
        sess.promote(name=casName)
        filehdl = upRes.casTable
        return(filehdl)
    
hr_cas = tbltoCAS(HR_df,"Heartrate")
sc_cas = tbltoCAS(SC_df,"Stepcount")

NOTE: Cloud Analytic Services made the uploaded file available as table HEARTRATE in caslib CASUSER(sasdemo).
NOTE: The table HEARTRATE has been created in caslib CASUSER(sasdemo) from binary data uploaded to Cloud Analytic Services.
NOTE: Cloud Analytic Services made the uploaded file available as table STEPCOUNT in caslib CASUSER(sasdemo).
NOTE: The table STEPCOUNT has been created in caslib CASUSER(sasdemo) from binary data uploaded to Cloud Analytic Services.


In [51]:
sess.table.tableinfo()

,Name,Rows,Columns,Encoding,CreateTimeFormatted,ModTimeFormatted,JavaCharSet,CreateTime,ModTime,Global,Repeated,View,SourceName,SourceCaslib,Compressed,Creator,Modifier
0,APPLEHEARTRATE,43219,5,utf-8,09Jun2017:13:31:36,09Jun2017:13:31:36,UTF8,1.812634e+09,1.812634e+09,1,0,0,,,0,sasdemo,
1,HEARTRATE,43219,5,utf-8,09Jun2017:14:04:00,09Jun2017:14:04:00,UTF8,1.812636e+09,1.812636e+09,1,0,0,,,0,sasdemo,
2,STEPCOUNT,21066,5,utf-8,09Jun2017:14:04:01,09Jun2017:14:04:01,UTF8,1.812636e+09,1.812636e+09,1,0,0,,,0,sasdemo,


## Now the table is in a public space in CAS, it is availble in VA